In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
# Klasör yapısını kontrol et ve etiketleri listele
directory = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'
classes = [i for i in os.listdir(directory) if '.' not in i]

In [13]:
label = []
path = []

# Dosya yollarını ve etiketlerini topla
for dir_name, _, filenames in os.walk(directory):
    for filename in filenames:
        if os.path.splitext(filename)[-1] == '.png':  # If filename contains .png
            if dir_name.split()[-1] != 'GT':  # If directory doesn't contain GT
                label.append(os.path.split(dir_name)[-1])  # Append the directory name to label
                path.append(os.path.join(dir_name, filename))  # Append all the png files to path of that directory

data = pd.DataFrame(columns=['path', 'label'])
data['path'] = path
data['label'] = label

In [14]:
def load_images(image_paths, target_size=(128, 128)):  # Boyutu 128x128 yap
    images = [img_to_array(load_img(img, target_size=target_size)) / 255.0 for img in image_paths]
    return np.array(images)

# Görüntüleri yükleme
X = load_images(data['path'].values)


In [15]:
# Etiketleri sayısal değerlere dönüştürme
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(data['label'].values)

# Sayısal etiketleri one-hot encoding yapma
y = to_categorical(y_encoded, num_classes=len(label_encoder.classes_))


In [16]:
# Eğitim, doğrulama ve test setlerine ayır
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [17]:
# Veri artırma uygulama
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [18]:
# ANN modelini tanımla
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),  # İlk konvolüsyon katmanı
    layers.MaxPooling2D(pool_size=(2, 2)),  # Max pooling
    layers.Conv2D(64, (3, 3), activation='relu'),  # İkinci konvolüsyon katmanı
    layers.MaxPooling2D(pool_size=(2, 2)),  # Max pooling
    layers.Conv2D(128, (3, 3), activation='relu'),  # Üçüncü konvolüsyon katmanı
    layers.MaxPooling2D(pool_size=(2, 2)),  # Max pooling
    layers.Flatten(),  # Düzleştirme
    layers.Dense(512, activation='relu'),  # İlk gizli katman
    layers.Dropout(0.5),  # Dropout katmanı
    layers.Dense(256, activation='relu'),  # İkinci gizli katman
    layers.Dropout(0.5),  # Dropout katmanı
    layers.Dense(len(set(label)), activation='softmax')  # Çıkış katmanı
])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
# Modeli derle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
# Modeli eğit
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    epochs=30)  # Epoch sayısını artırdık


Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


197/197 ━━━━━━━━━━━━━━━━━━━━ 122s 605ms/step - accuracy: 0.1358 - loss: 2.1369 - val_accuracy: 0.3252 - val_loss: 1.6516
Epoch 2/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 120s 606ms/step - accuracy: 0.3668 - loss: 1.5450 - val_accuracy: 0.5193 - val_loss: 1.2276
Epoch 3/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 142s 607ms/step - accuracy: 0.5500 - loss: 1.0988 - val_accuracy: 0.6741 - val_loss: 0.8075
Epoch 4/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 121s 608ms/step - accuracy: 0.6683 - loss: 0.8619 - val_accuracy: 0.8437 - val_loss: 0.4497
Epoch 5/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 121s 608ms/step - accuracy: 0.7662 - loss: 0.6082 - val_accuracy: 0.8644 - val_loss: 0.3438
Epoch 6/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 142s 606ms/step - accuracy: 0.8041 - loss: 0.5379 - val_accuracy: 0.8481 - val_loss: 0.3690
Epoch 7/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 120s 605ms/step - accuracy: 0.8463 - loss: 0.4151 - val_accuracy: 0.9259 - val_loss: 0.2086
Epoch 8/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 121s 608ms/step - accuracy: 0.8549 - loss: 0.41

In [21]:
# Test setinde modeli değerlendir
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.0505, Test Accuracy: 0.9793
